In [7]:
# Cell 0: Create credentials file (Run this ONCE)
import os

# Navigate to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
env_path = os.path.join(project_root, '.env')

# Your Neo4j credentials
neo4j_credentials = """NEO4J_URI=neo4j+s://2dad9bcd.databases.neo4j.io
NEO4J_USER=neo4j
NEO4J_PASSWORD=tZxE7BCzFpktLV5PxI2nXtxuXIGVEjp1Z0dBZ8Z57vs
"""

with open(env_path, 'w') as f:
    f.write(neo4j_credentials)

print(f"✅ Credentials saved to: {env_path}")
print(f"✅ File exists: {os.path.exists(env_path)}")

✅ Credentials saved to: d:\1\Knowledge Graph\knowledge_graph_course\.env
✅ File exists: True


In [9]:
# Cell 1: Setup Neo4j Connection
"""
Cypher Basics - Querying Property Graphs
========================================
We'll query the Movies database using Cypher (Neo4j's query language)
"""


from neo4j import GraphDatabase
import pandas as pd
import sys
sys.path.append('..')
from scripts.helper_functions import QueryHelper, load_neo4j_credentials

# Load credentials
creds = load_neo4j_credentials()
if not creds:
    raise Exception("Run setup_neo4j.py first to configure credentials!")

# Connect to Neo4j
driver = GraphDatabase.driver(
    creds['NEO4J_URI'],
    auth=(creds['NEO4J_USER'], creds['NEO4J_PASSWORD'])
)

helper = QueryHelper()

# Verify connection
with driver.session() as session:
    result = session.run("RETURN 'Connected!' as message")
    print(f"✅ {result.single()['message']}")
    
print("Ready to query the Movies database!")

❌ .env file not found. Run setup_neo4j.py first!


Exception: Run setup_neo4j.py first to configure credentials!